# 网络中的网络（NiN）
前⼏节介绍的LeNet、 AlexNet和VGG在设计上的共同之处是：先以由卷积层构成的模块充分抽取
空间特征，再以由全连接层构成的模块来输出分类结果。其中， AlexNet和VGG对LeNet的改进主
要在于如何对这两个模块加宽（增加通道数）和加深。本节我们介绍⽹络中的⽹络（NiN） [1]。它
提出了另外⼀个思路，即串联多个由卷积层和“全连接”层构成的小⽹络来构建⼀个深层⽹络。
## NiN块
我们知道，卷积层的输⼊和输出通常是四维数组（样本，通道，⾼，宽），而全连接层的输⼊和输出则
通常是⼆维数组（样本，特征）。如果想在全连接层后再接上卷积层，则需要将全连接层的输出变换
为四维。回忆在[“多输⼊通道和多输出通道”](5.3channels.ipynb) ⼀节⾥介绍的1×1卷积层。它可以看成全连接层，其
中空间维度（⾼和宽）上的每个元素相当于样本，通道相当于特征。因此， NiN使⽤1×1卷积层来替
代全连接层，从而使空间信息能够⾃然传递到后⾯的层中去。图5.7对⽐了NiN同AlexNet和VGG等
⽹络在结构上的主要区别

![左图是AlexNet和VGG的网络结构局部，右图是NiN的网络结构局部](../img/nin.svg)

NiN块是NiN中的基础块。它由⼀个卷积层加两个充当全连接层的1 × 1卷积层串联而成。其中第
⼀个卷积层的超参数可以⾃⾏设置，而第⼆和第三个卷积层的超参数⼀般是固定的。

In [2]:
import d2lzh as d2l
from mxnet import gluon, init, nd
from mxnet.gluon import nn
def nin_block(num_channels, kernel_size, strides, padding):
    blk = nn.Sequential()
    blk.add(nn.Conv2D(num_channels, kernel_size, strides, padding, activation='relu'),
        nn.Conv2D(num_channels, kernel_size=1, activation='relu'),
        nn.Conv2D(num_channels, kernel_size=1, activation='relu'))
    return blk

## NiN模型
NiN是在AlexNet问世不久后提出的。它们的卷积层设定有类似之处。 NiN使⽤卷积窗口形状分别
为11 × 11、 5 × 5和3 × 3的卷积层，相应的输出通道数也与AlexNet中的⼀致。每个NiN块后接⼀
个步幅为2、窗口形状为3 × 3的最⼤池化层。

除使⽤NiN块以外， NiN还有⼀个设计与AlexNet显著不同： NiN去掉了AlexNet最后的3个全连接
层，取而代之地， NiN使⽤了输出通道数等于标签类别数的NiN块，然后使⽤全局平均池化层对
每个通道中所有元素求平均并直接⽤于分类。这⾥的全局平均池化层即窗口形状等于输⼊空间维
形状的平均池化层。 NiN的这个设计的好处是可以显著减小模型参数尺⼨，从而缓解过拟合。然
而，该设计有时会造成获得有效模型的训练时间的增加。

In [3]:
net = nn.Sequential()
net.add(nin_block(96, kernel_size=11, strides=4, padding=0),
    nn.MaxPool2D(pool_size=3, strides=2),
    nin_block(256, kernel_size=5, strides=1, padding=2),
    nn.MaxPool2D(pool_size=3, strides=2),
    nin_block(384, kernel_size=3, strides=1, padding=1),
    nn.MaxPool2D(pool_size=3, strides=2), nn.Dropout(0.5),
    # 标签类别数是10
    nin_block(10, kernel_size=3, strides=1, padding=1),
    # 全局平均池化层将窗⼝形状⾃动设置成输⼊的⾼和宽
    nn.GlobalAvgPool2D(),
    # 将四维的输出转成⼆维的输出，其形状为(批量⼤⼩, 10)
    nn.Flatten())

我们构建⼀个数据样本来查看每⼀层的输出形状。

In [4]:
X = nd.random.uniform(shape=(1, 1, 224, 224))
net.initialize()
for layer in net:
    X = layer(X)
    print(layer.name, 'output shape:\t', X.shape)

sequential1 output shape:	 (1, 96, 54, 54)
pool0 output shape:	 (1, 96, 26, 26)
sequential2 output shape:	 (1, 256, 26, 26)
pool1 output shape:	 (1, 256, 12, 12)
sequential3 output shape:	 (1, 384, 12, 12)
pool2 output shape:	 (1, 384, 5, 5)
dropout0 output shape:	 (1, 384, 5, 5)
sequential4 output shape:	 (1, 10, 5, 5)
pool3 output shape:	 (1, 10, 1, 1)
flatten0 output shape:	 (1, 10)


## 获取数据和训练模型
我们依然使⽤Fashion-MNIST数据集来训练模型。 NiN的训练与AlexNet和VGG的类似，但这⾥使
⽤的学习率更⼤。

In [5]:
lr, num_epochs, batch_size, ctx = 0.1, 5, 128, d2l.try_gpu()
net.initialize(force_reinit=True, ctx=ctx, init=init.Xavier())
trainer = gluon.Trainer(net.collect_params(), 'sgd', {'learning_rate': lr})
train_iter, test_iter = d2l.load_data_fashion_mnist(batch_size, resize=224)
d2l.train_ch5(net, train_iter, test_iter, batch_size, trainer, ctx, num_epochs)

## ⼩结
- NiN重复使⽤由卷积层和代替全连接层的1 × 1卷积层构成的NiN块来构建深层⽹络。
- NiN去除了容易造成过拟合的全连接输出层，而是将其替换成输出通道数等于标签类别数
的NiN块和全局平均池化层。
- NiN的以上设计思想影响了后⾯⼀系列卷积神经⽹络的设计。